# Versuche für Filter-Stage

In [9]:
import pdal 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json

from interessant import *

In [10]:
interessant.keys()

dict_keys(['Einfach', 'Weiche', 'Ende', 'Bahnübergang', 'Bahnübergang 2', 'Zug run 14 A', 'Zug run 14 B', 'Zug run 24 A', 'Zug run 24 B', 'Gebäude', 'Gleis hohe Intensität', 'Feldweg', 'Feld', 'Weiche A', 'Weiche B', 'Zaun', 'Straße', 'Kante in Graben', 'Unterirdischer Bhf', 'Fangschiene Tunnel', 'Fangschiene Tunnel 2', 'Gleis weit abseits', 'Komische Linie', 'Kabeltöpfe', 'Güterzug', 'Güterzug Ende', 'Betondeckel', 'Bahnsteig', 'Bahnsteig Ende', 'Ding neben Gleis', 'Wände', 'Viele Gleise'])

In [11]:
run = run24
filename = interessant["Einfach"]
filename = os.path.join(run24, filename)
os.path.exists(filename)

True

In [12]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]
points.dtype # Column names and types

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [13]:
points['Classification'] = 0 # Unclassified

In [14]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

intensity = points['Intensity']
intensity_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colormap = plt.get_cmap("viridis")
intensity_colors = colormap(intensity_normalized)
intensity_colors = intensity_colors[:, :3]

# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
xyz -= offset

In [15]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(intensity_colors)
# pcd.colors = o3d.utility.Vector3dVector(rgb)

In [16]:
o3d.visualization.draw_geometries([pcd])

# Teile in säulenartige Voxel, um darin z zu untersuchen

Wie Oude Elberink et al. 2013 und auch in Oude Elberink und Kh. 2015 und in Kononen et al. 2024

In [17]:
voxel_size = 1.0
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([11.99997543, 11.99998693, 13.12408   ]),
 array([-13.00002457, -13.00001307,  -3.99652   ]))

In [18]:
my_classes = {
    "Unclassified": 0,
    "High Points": 13,   
    "No clearance": 14,
    "Missing Rail": 15,
    "Low Points": 16,
    "Rail": 20,
}

color_map = {
    13: [0.3, 0.3, 0.3],
    14: [0.5, 0.5, 0.5], 
    15: [0, 0, 1],
    16: [0, 1, 0],
    20: [1, 0, 0],
}

In [19]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [20]:
voxels  

array([[13.     , 22.     , 11.86328],
       [14.     , 19.     , 12.22228],
       [13.     , 22.     , 11.89178],
       ...,
       [21.     , 24.     ,  1.27228],
       [17.     , 22.     ,  0.67448],
       [19.     , 18.     , -1.37662]])

In [21]:
len(voxels) 

1803127

In [22]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [23]:
# Umwandeln der Listen in Arrays (optional)
# for key in voxel_dict:
#     voxel_dict[key] = np.array(voxel_dict[key])

In [24]:
viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

In [25]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < 100:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    thresh = 2
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        candidates_top = np.percentile(z_values[mask], 99.5)

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < 0.3 * len(z_values):
                points['Classification'][indices[mask]] = my_classes["Rail"]
                points['Classification'][indices[~mask]] = my_classes["Low Points"]
            else:
                points['Classification'][indices] = my_classes["Missing Rail"]


        else:
            # There are no points above ground 
            points['Classification'][indices] = my_classes["Missing Rail"]


        # Class for high points
        mask = (z_values >= ground_level + 4.5)
        points['Classification'][indices[mask]] = my_classes["High Points"]


    else:
        # Discard voxel
        # Set color of all points in voxel to grey
        points['Classification'][indices] = my_classes["No clearance"]




Oude E. & K. nehmen die 98. Percentile, vermutlich wegen Noise. Zum Teil gehen mir Rail-Punkte verloren, wenn das Gleis nur in der Ecke des Voxels vorkommt. In vielen Fällen ist Problem bei 99.9 Percentile behoben

In [26]:
class_colors = np.zeros_like(rgb)
for class_value, color in color_map.items():
    class_colors[points['Classification'] == class_value] = color

In [27]:
# Optional Schwellenwert für Intensität
intensity_threshold = 14500

mask = (points["Classification"] == my_classes["Rail"]) & (points["Intensity"] > intensity_threshold)
class_colors[mask] = [0.9, 0.5, 0]

In [28]:
pcd.colors = o3d.utility.Vector3dVector(class_colors)
o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom)

In [29]:
out_pipeline = pdal.Writer("test.laz").pipeline(points)
out_pipeline.execute()

1803127